In [ ]:
!pip install nbformat lxml

from lxml import etree

In [ ]:
# Globals
from common import _check_task_size, data_dir, metadata_dir, set_id, output_file, unpack_metadata, unzip_file, zip_file
# Getters
from common import get_liner_output_files_ch3, get_date_from_metadata, get_resource_ids_from_metadata, get_resource_file, get_spellchecked_resources_ch3

## Named Entity Recognition with Liner2

In this section we will present how to use Europeana bibliographic resources with Named Entity Recognition (NER) tool for Polish using [lpmn_client](https://wiki.clarin-pl.eu/en/nlpws/lpmn_client) and [Liner2](https://github.com/CLARIN-PL/Liner2). Due to the server side limitation, we ensure 2M limit on size on the task with function defined below. 


In [ ]:
"""
    Install lpmn client and import it
"""
!pip install -i https://pypi.clarin-pl.eu lpmn_client

from lpmn_client import download_file, upload_file
from lpmn_client import Task

### Spellchecking with lpmn client

OCR output tends to contains spelling mistakes that adds noise to the data. Below we show how to use lpmn clinet to specify task pipeline in order to obtain spell-checked textual data. 

In [ ]:
"""
    Function for tasking lpmn client with Liner2 NER pipeline with task size control 
"""

def lpmn_client_task(resources, task, names=[]):
    """
        Wrap over CLARIN-PL lpmn client with control of the task size in order to avoid jamming the task queue on the server side
        
        :param list resources: list of paths to the resources to be processed
        :param str task: string defining pipeline, e.g. "speller2" or ""
        :param list names: optional list of names for output files, has to be same length as resources
        :returns list: list of paths to the output zip files
    """
    
    # Size check
    _check_task_size(resources)
    # Upload reasources to task queue
    job_ids = [upload_file(resource_file) for resource_file in resources]
    # Specify pipeline 
    t = Task(task)
    # Run uploaded tasks with pipeline
    output_file_ids = [t.run(job_id, verbose=True) for job_id in job_ids]
    if names:
        output = [download_file(output_file_id, output_file, f"{filename}.zip") 
                         for output_file_id, filename in zip(output_file_ids, names)]
    else:
        output = [download_file(output_file_id, output_file, f"{os.path.basename(resource)}.zip") 
                         for output_file_id, resource in zip(output_file_ids, resources)]
    return output

In [ ]:
"""
    Let's select resources. We will use Izraelita newspapers from 1910-1913 to investigate most frequent Named Entities over the years on January. 
"""

# Prepare metadata of the collection
unpack_metadata(set_id, metadata_dir)
izraelita_metadata_files = [f"{metadata_dir}/Izraelita_1911.xml",
                            f"{metadata_dir}/Izraelita_1912.xml",
                            f"{metadata_dir}/Izraelita_1913.xml",
                           ]

# Use getters to obtain date and id from metadata
izraelita_metadata_trees = [etree.parse(izraelita_metadata_file) for izraelita_metadata_file in izraelita_metadata_files]
ids = [get_resource_ids_from_metadata(izraelita_metadata_tree) for izraelita_metadata_tree in izraelita_metadata_trees]
dates = [get_date_from_metadata(izraelita_metadata_tree) for izraelita_metadata_tree in izraelita_metadata_trees]

# Get all issues from January
ids_dates_january = []
for _ids, _dates in zip (ids, dates):
    year = []
    for _id, date in zip(_ids, _dates):
        if date.month==1:
            year.append((_id, date))
    ids_dates_january.append(year)

# Map id to reasource
resources_dates = [(get_resource_file(_id), date) for year in ids_dates_january for _id, date in year ]
print(resources_dates)

In [ ]:
"""
    Let's spell check selected resources
"""

# Safety check of the size of all resources meant for the task
_check_task_size([resource for resource, _ in resources_dates])

# For offline run uncomment block below and comment last line
# for resource, date in resources_dates:
#     lpmn_client_task([resource], "speller2", [f"{date}_speller"])
# speller2_output_file_paths = [unzip_file(f"{output_file}/{date}_speller.zip")[0] for _, date in resources_dates]

speller2_output_file_paths = get_spellchecked_resources_ch3()
speller2_output_file_paths

In [ ]:
# Safety check of the size of all resources meant for the task
_check_task_size(speller2_output_file_paths)

for resource, (_, date) in zip(speller2_output_file_paths, resources_dates):
    lpmn_client_task([resource],'any2txt|wcrft2|liner2({"model":"top9"})', [f"{date}_liner"])
    


In [ ]:
liner2_output_file_paths = [unzip_file(f"{output_file}/{date}_liner.zip")[0] for _, date in resources_dates]
liner2_output_file_paths

In [ ]:
"""
    Function for extracting annotations from Liner2 xml output
"""


def liner2_xml_to_annotation(xml_tree):
    """
        Converts xml doc into list of annotations and tokens
        
        :param str path_to_xml: path to .xml Liner2 output file
        :returns list: list of tuples (annotation_type, [tokens])
    """

    sentences = xml_tree.xpath("//sentence")
    annotated_tokens = [sentence.xpath("./tok[./ann!=0]") for sentence in sentences]
    # Prune empty lists
    annotated_tokens = filter(lambda x: True if x else False, annotated_tokens)
    annotated_tokens = [_chain_annotations(sentence) for sentence in annotated_tokens]
    return annotated_tokens
        
def _chain_annotations(sentence: list):
    annotation_heads = [token.xpath("./ann[@head]") for token in sentence]
    for token in annotation_heads:
        for annotation_head in token:
            annotation_channel = annotation_head.xpath("./text()")[0]
            annotation_type = annotation_head.get("chan")
            annotation_tokens = [token.xpath("./lex/base/text()")[0] for token in sentence if token.xpath(f"./ann[text()={annotation_channel}]")]
    return annotation_type, annotation_tokens

In [ ]:
"""
    Parse NER output and investigate 10 most common named entities by years
"""

from collections import Counter
import os

named_entities_counts = {}
for _, date in resources_dates:
    for root, _, files in os.walk(f"{output_file}/{date}_liner"):
        for filename in files:
            path_to_annotated_output = os.path.join(root, filename)
            xml_tree = etree.parse(path_to_annotated_output)
            # Get stats
            token_nb = sum([1 for _ in xml_tree.xpath("//tok")])
            annotation_list = liner2_xml_to_annotation(xml_tree)
            annotation_counts = Counter(f"{annotation_type}|{' '.join(annotation_tokens)}" for annotation_type, annotation_tokens in annotation_list)
            if date.year in named_entities_counts.keys():
                named_entities_counts[date.year].append((token_nb, annotation_counts))
            else:
                named_entities_counts[date.year] = [(token_nb, annotation_counts)]
most_common_overall = {year: (sum([token_nb for token_nb, _ in named_entities_counts[year]]), 
                              sum([counter for _, counter in named_entities_counts[year]], Counter()).most_common(10)
                             ) for year in named_entities_counts.keys()}

In [ ]:
most_common_overall